In [1]:
import subprocess
subprocess.run(['pip', 'install', '-q', '-U', 'google-genai', 'openai', 'python-dotenv', 'gradio', 'requests'])

CompletedProcess(args=['pip', 'install', '-q', '-U', 'google-genai', 'openai', 'python-dotenv', 'gradio', 'requests'], returncode=0)

In [2]:
import os
import requests
import json
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
import gradio as gr
import subprocess
import platform
import tempfile
import re
import shutil

In [3]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')

In [4]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o-mini"
GEMINI_MODEL = "gemini-2.5-flash"

In [5]:
system_message = "You are expert specializing in Test-Driven Development (TDD)."
system_message += "Your primary task is to write high-quality, comprehensive unit tests for the provided code and to add clear, helpful comments where necessary to explain logic, complex sections, or edge cases."
system_message += """ Key Guidelines for Output Generation:
1. Programming Language: 
    *You must automatically detect the programming language of the input code and generate tests and comments strictly in that language.
2. Unit Tests:
    *Generate unit tests using the most idiomatic and standard testing framework for the detected language (e.g., pytest or unittest for Python, Jest for JavaScript, JUnit for Java, Go testing for Go).
    *Tests must cover all logical branches, edge cases, and typical use scenarios of the input code.
    *Use descriptive, clear test names.
3. Comments:
    *Add comments only where they genuinely improve readability or understanding.
    *Use standard documentation practices for the detected language (e.g., docstrings, Javadoc, TSDoc) for functions/methods and classes.
    *Crucially, include a brief comment explaining the purpose of each unit test method (i.e., what specific behavior it verifies).
4. Content Order and Format (Strict Requirements for Automation):
    *Order: Unit tests must be presented first, followed by the commented version of the original source code.
    *Execution: If the detected language/framework supports it, the output should include the necessary execution statement (e.g., if __name__ == "__main__": in Python, or the main function if applicable) so that the file is easily executable.
    *No External Text: The output must not include any introductory or concluding text, including initial or final Markdown code fences (```), greetings, or explanations. The entire response should be the raw, runnable code structure.
    *Inclusion: The base/original source code must be included in the response (usually after the unit tests, in its commented form).
5. For C++, you must not use any external testing frameworks like GTest. Instead, write a main() function that uses <iostream> and <cassert> to run tests. The program must exit(1) or return 1 if any assertion fails.
6. For JavaScript, please generate tests using CommonJS modules (require) and not ES Modules (import).
"""

In [6]:
def user_prompt_for(code):
    user_prompt = "Please analyze the programming language of the following code block, and then generate comprehensive unit tests and add necessary comments for it"
    user_prompt += """Requirements:
                        *Testing Framework (Optional): [If you have a preference, specify the framework, e.g., 'Use pytest', 'Use Jest', 'Use JUnit'].
                        *Focus (Optional): [Specify a particular area of focus, e.g., 'Pay special attention to asynchronous operations', 'Ensure all database interactions are mocked'].\n\n"""
    user_prompt += code
    return user_prompt

In [7]:
def messages_for(code):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(code)}
    ]

In [8]:
example1 = """
def calculate_discounted_price(original_price: float, discount_percentage: float) -> float:

    if original_price <= 0:
        raise ValueError("Cena oryginalna musi być dodatnia.")
    
    if not (0 <= discount_percentage <= 100):
        raise ValueError("Procent zniżki musi być między 0 a 100.")

    discount_factor = discount_percentage / 100
    discounted_price = original_price * (1 - discount_factor)
    
    # Zaokrąglenie do dwóch miejsc po przecinku
    return round(discounted_price, 2)

# Funkcja pomocnicza
def is_prime(n: int) -> bool:
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True
"""

In [9]:
example2 = """
public class BankAccount {
    private double balance;

    public BankAccount(double initialBalance) {
        if (initialBalance < 0) {
            throw new IllegalArgumentException("Początkowe saldo nie może być ujemne.");
        }
        this.balance = initialBalance;
    }

    public double deposit(double amount) {
        if (amount <= 0) {
            throw new IllegalArgumentException("Kwota wpłaty musi być dodatnia.");
        }
        balance += amount;
        return balance;
    }

    public double withdraw(double amount) {
        if (amount <= 0) {
            throw new IllegalArgumentException("Kwota wypłaty musi być dodatnia.");
        }
        if (balance < amount) {
            throw new IllegalStateException("Niewystarczające środki.");
        }
        balance -= amount;
        return balance;
    }

    public double getBalance() {
        return balance;
    }
}
"""

In [10]:
def stream_tests_gpt(code):
    reply = ""
    try:
        stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(code), stream=True)
        for chunk in stream:
            fragment = chunk.choices[0].delta.content or ""
            reply += fragment
    except Exception as e:
        reply = f"[ERROR] GPT call failed: {e}"
    return reply

In [11]:
def stream_tests_gemini(code):
    reply = ""
    try:
        client = genai.Client()

        stream = client.models.generate_content_stream(
            model=GEMINI_MODEL,
            contents=user_prompt_for(code),
            config=types.GenerateContentConfig(system_instruction=system_message)
        )

        for i, chunk in enumerate(stream):
            fragment = ""
            try:
                fragment = chunk.text
            except ValueError:
                pass
            reply += fragment

    except Exception as e:
        reply = f"[ERROR] {e}"
    return reply

In [12]:
def select_model(code, model):
    if not code or code.strip() == "":
        return "No code provided."
    if model == "GPT":
        return stream_tests_gpt(code)
    elif model == "Gemini":
        return stream_tests_gemini(code)
    else:
        return "Invalid model selected."

In [13]:
def select_sample_code(sample_code):
    if sample_code=="example1":
        return example1
    elif sample_code=="example2":
        return example2
    else:
        return ""

In [14]:
JUNIT_JAR = "junit-platform-console-standalone-1.10.3.jar"
JUNIT_URL = "https://repo1.maven.org/maven2/org/junit/platform/junit-platform-console-standalone/1.10.3/junit-platform-console-standalone-1.10.3.jar"

In [15]:
def download_junit():
    if not os.path.exists(JUNIT_JAR):
        try:
            r = requests.get(JUNIT_URL)
            r.raise_for_status()
            with open(JUNIT_JAR, 'wb') as f:
                f.write(r.content)
            print(f"Successfully downloaded {JUNIT_JAR}")
        except requests.RequestException as e:
            print(f"Error downloading JUnit: {e}")
            return False
    return True

In [16]:
VISUAL_STUDIO_2022_TOOLS = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2022\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"

simple_cpp = """
#include <iostream>

int main() {
    std::cout << "Hello";
    return 0;
}
"""

In [17]:
def get_cpp_compiler():
    compilers = []
    my_platform = platform.system()
    if my_platform == 'Windows':
        compilers = [
            ('g++', ['g++', '-o', '{filename_base}', '{filename_base}.cpp']),
            ('clang++', ['clang++', '-o', '{filename_base}', '{filename_base}.cpp']),
            ('cl', ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", '{filename_base}.cpp'])
        ]
    else:
        compilers = [
            ('g++', ['g++', '-o', '{filename_base}', '{filename_base}.cpp']),
            ('clang++', ['clang++', '-o', '{filename_base}', '{filename_base}.cpp'])
        ]
    
    with tempfile.TemporaryDirectory() as tmpdir:
        simple_file = os.path.join(tmpdir, 'simple.cpp')
        with open(simple_file, 'w') as f:
            f.write(simple_cpp)
        
        for compiler_name, compile_template in compilers:
            try:
                subprocess.run([compiler_name, '--version'], check=True, capture_output=True)
                
                compile_cmd = [arg.replace('{filename_base}', 'simple') for arg in compile_template]
                subprocess.run(compile_cmd, cwd=tmpdir, check=True, capture_output=True)
                
                exe_name = 'simple.exe' if my_platform == 'Windows' else 'simple'
                exe_path = os.path.join(tmpdir, exe_name)
                result = subprocess.run([exe_path], cwd=tmpdir, capture_output=True, text=True, check=True)
                
                if "Hello" in result.stdout:
                    return [my_platform, compiler_name, compile_template]
            except Exception:
                pass
    
    return [my_platform, "Unavailable", []]

In [18]:
def detect_language_llm(tests, model):
    detection_prompt = (
        "Detect the programming language of the following code. "
        'Output in JSON format like: {"language": "python", "extension": "py"}\n'
        "Supported: python (py), java (java), cpp (cpp), csharp (cs), javascript (js).\n\n"
        "Code:\n"
        + (tests[:2000])  # truncated
    )
    raw = ""
    try:
        if model == "GPT":
            resp = openai.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[{"role": "user", "content": detection_prompt}]
            )
            raw = resp.choices[0].message.content
        elif model == "Gemini":
            client = genai.Client()
            resp = client.models.generate_content(
                model=GEMINI_MODEL,
                contents=detection_prompt
            )
            try:
                raw = resp.text
            except ValueError as ve:  # Handle safety blocks
                print(f"Gemini response blocked: {ve}")
                raw = ""
        else:
            raw = ""
    except Exception as e:
        print(f"Error in LLM language detection: {e}")
        raw = ""
    
    try:
        parsed = json.loads(raw)
        return parsed.get("language", "unknown"), parsed.get("extension", "")
    except Exception:
        if "def " in tests and ":" in tests:
            return "python", "py"
        if "public class" in tests or "import java" in tests:
            return "java", "java"
        if "#include" in tests or "std::" in tests:
            return "cpp", "cpp"
        if "using System" in tests or "namespace" in tests:
            return "csharp", "cs"
        if "function " in tests or "console.log" in tests or "module.exports" in tests:
            return "javascript", "js"
        return "unknown", ""

In [19]:
def run_python_tests(filename, tmpdir, tests):
    if 'import pytest' in tests or 'pytest' in tests:
        try:
            subprocess.run(['pytest', '--version'], check=True, capture_output=True)
            cmd = ['pytest', filename]
        except Exception:
            return "pytest is not installed. Please install pytest or use unittest."
    else:
        cmd = ['python', filename]
    try:
        result = subprocess.run(cmd, cwd=tmpdir, capture_output=True, text=True, check=False)
        output = result.stdout + (('\nErrors:\n' + result.stderr) if result.stderr else '')
    except Exception as e:
        output = str(e)
    return output

In [20]:
JAVA_HOME = r"C:\Users\....." #you have to put here your JDK's bin directory PATH

In [21]:
def run_java_tests(filename, tmpdir):
    javac_path = os.path.join(JAVA_HOME, 'bin', 'javac.exe')
    java_path = os.path.join(JAVA_HOME, 'bin', 'java.exe')
    
    try:
        subprocess.run([javac_path, '-version'], check=True, capture_output=True)
        subprocess.run([java_path, '-version'], check=True, capture_output=True)
    except Exception as e:
        return f"Java compiler or runtime not available: {str(e)}"
    
    if not download_junit():
        return "Failed to download JUnit."
    cmd_compile = [javac_path, '-cp', JUNIT_JAR, filename]
    result = subprocess.run(cmd_compile, cwd=tmpdir, capture_output=True, text=True)
    if result.returncode != 0:
        return result.stderr
    with open(filename, 'r') as f:
        content = f.read()
        match = re.search(r'public\s+class\s+(\w+)', content)
        if match:
            class_name = match.group(1)
        else:
            return "Could not find public test class name."
    
    cmd_run = [java_path, '-jar', os.path.abspath(JUNIT_JAR), '--class-path', '.', '--select-class', class_name]
    result = subprocess.run(cmd_run, cwd=tmpdir, capture_output=True, text=True)
    output = result.stdout + (('\nErrors:\n' + result.stderr) if result.stderr else '')
    return output

In [22]:
def run_cpp_tests(filename_base, tmpdir):
    global compiler_cmd
    if compiler_cmd[1] == "Unavailable":
        return "C++ compiler not available."
    compile_template = compiler_cmd[2]
    compile_cmd = [arg.replace('{filename_base}', filename_base) for arg in compile_template]
    result = subprocess.run(compile_cmd, cwd=tmpdir, capture_output=True, text=True)
    if result.returncode != 0:
        return result.stderr
    exe_name = f"{filename_base}.exe" if platform.system() == "Windows" else filename_base
    cmd_run = [os.path.join('.', exe_name)]
    result = subprocess.run(cmd_run, cwd=tmpdir, capture_output=True, text=True)
    output = result.stdout + (('\nErrors:\n' + result.stderr) if result.stderr else '')
    return output

In [23]:
def run_csharp_tests(filename, tmpdir):
    try:
        subprocess.run(['dotnet', '--version'], check=True, capture_output=True)
    except Exception:
        return ".NET SDK not available."
    project_dir = os.path.join(tmpdir, 'TestProject')
    os.mkdir(project_dir)
    result = subprocess.run(['dotnet', 'new', 'xunit', '--no-restore'], cwd=project_dir, capture_output=True, text=True)
    if result.returncode != 0:
        return result.stderr
    default_test_file = os.path.join(project_dir, 'UnitTest1.cs')
    try:
        os.remove(default_test_file)
    except FileNotFoundError:
        pass
    shutil.copy(filename, os.path.join(project_dir, 'Tests.cs'))
    result = subprocess.run(['dotnet', 'test', '--no-restore'], cwd=project_dir, capture_output=True, text=True)
    output = result.stdout + (('\nErrors:\n' + result.stderr) if result.stderr else '')
    return output

In [24]:
def run_javascript_tests(filename, tmpdir):
    try:
        subprocess.run(['jest', '--version'], check=True, capture_output=True)
    except Exception:
        return "Jest is not installed. Please install Jest globally or locally."
    cmd = ['jest', filename]
    result = subprocess.run(cmd, cwd=tmpdir, capture_output=True, text=True)
    output = result.stdout + (('\nErrors:\n' + result.stderr) if result.stderr else '')
    return output

In [25]:
def run_unit_tests(tests, model):
    language, ext = detect_language_llm(tests or "", model)
    if language == 'unknown':
        return "Unknown programming language detected."
    with tempfile.TemporaryDirectory() as tmpdir:
        filename_base = 'test_code'
        filename = os.path.join(tmpdir, f"{filename_base}.{ext}")
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(tests)
        if language == 'python':
            return run_python_tests(filename, tmpdir, tests)
        elif language == 'java':
            return run_java_tests(filename, tmpdir)
        elif language == 'cpp':
            return run_cpp_tests(filename_base, tmpdir)
        elif language == 'csharp':
            return run_csharp_tests(filename, tmpdir)
        elif language == 'javascript':
            return run_javascript_tests(filename, tmpdir)
        else:
            return f"Language '{language}' not supported by runner."

In [26]:
compiler_cmd = get_cpp_compiler()

with gr.Blocks() as ui:
    with gr.Row():
        code_to_test = gr.Textbox(label="Code to test", lines=20)
        sample_code = gr.Radio(["example1", "example2", "Your own code"], label="Sample code", value="example1")
        tests = gr.Textbox(label="Unit tests", lines=20)
    with gr.Row():
        selected_model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
        generate_tests = gr.Button("Generate unit tests")
        run_tests = gr.Button("Run unit tests")
    with gr.Row():
        test_result = gr.Textbox(label="Unit tests results", lines=15)
    with gr.Row():
        architecture = gr.Radio([compiler_cmd[0]], label="Architecture", interactive=False, value=compiler_cmd[0])
        compiler = gr.Radio([compiler_cmd[1]], label="Compiler", interactive=False, value=compiler_cmd[1])

    # wiring
    sample_code.change(lambda v: example1 if v == "example1" else (example2 if v == "example2" else ""), inputs=[sample_code], outputs=[code_to_test])
    generate_tests.click(select_model, inputs=[code_to_test, selected_model], outputs=[tests])
    run_tests.click(run_unit_tests, inputs=[tests, selected_model], outputs=[test_result])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
